# Enforcement of Immigration Legislation

Eurostat - [Asylum and Managed Migration team](ESTAT-AMM-STATISTICS@ec.europa.eu)

27 May 2021

Welcome to this *statistics coded* notebook. This notebook developed with Rmd will allow to reproduce the graphics presented in the Statistics Explained article on [Enforcement of Immigration Legislation statistics](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Enforcement_of_immigration_legislation_statistics). 


<center>
<img alt="" src="https://ec.europa.eu/eurostat/statistics-explained/images/thumb/e/e0/Non-EU_citizens_subject_to_immigration_law_enforcement%2C_2019_number_MI20.jpg/500px-Non-EU_citizens_subject_to_immigration_law_enforcement%2C_2019_number_MI20.jpg" decoding="async" width="500" height="500" class="thumbimage" srcset="https://ec.europa.eu/eurostat/statistics-explained/images/thumb/e/e0/Non-EU_citizens_subject_to_immigration_law_enforcement%2C_2019_number_MI20.jpg/750px-Non-EU_citizens_subject_to_immigration_law_enforcement%2C_2019_number_MI20.jpg 1.5x, https://ec.europa.eu/eurostat/statistics-explained/images/thumb/e/e0/Non-EU_citizens_subject_to_immigration_law_enforcement%2C_2019_number_MI20.jpg/1000px-Non-EU_citizens_subject_to_immigration_law_enforcement%2C_2019_number_MI20.jpg 2x" />
</center>


This article presents indicators on the enforcement of immigration legislation. It provides statistics on: third country or non-European Union (EU) citizens who were refused entry at the external borders of the EU-27<sup>1</sup>; non-EU citizens who were illegally present on the territory of an EU Member State; and non-EU citizens who were ordered to leave the territory of an EU Member State<sup>2</sup>. Each of these indicators can be regarded as an official record of persons subject to the enforcement of EU immigration legislation, providing a general overview of the outcomes of territorial surveillance and control procedures.

<sup>1</sup> EU aggregates are computed as the sum of the national statistics available for the EU Member States. It is possible that the statistics for the EU involve some double counting of individuals if they are found to be illegally present in more than one Member State.

<sup>2</sup> Statistics on the enforcement of immigration legislation refer to the concept of external borders for all EU Member States and EFTA countries, even if some of these are not in the Schengen area. The external borders of the Schengen area do not coincide with the external borders of the EU Member States due to: opt-outs for Ireland from the Schengen area; Bulgaria, Croatia, Cyprus and Romania are not yet members of the Schengen area; Iceland, Liechtenstein, Norway and Switzerland are part of the Schengen area but are not members of the EU.

In [2]:
library(data.table)
library(restatapi)
library(plotly)
library(tmap)
library(tidyverse)
library(DT)
#library(formatR)
library(giscoR)

In [3]:
ref_tot <- setDT(get_eurostat_data("migr_eirfs"))
app_tot<- setDT(get_eurostat_data("migr_eipre"))
ord_tot <- setDT(get_eurostat_data("migr_eiord"))
ret_tot <- setDT(get_eurostat_data("migr_eirtn"))
ret_type <- setDT(get_eurostat_data("migr_eirt_vol"))
ret_ass <- setDT(get_eurostat_data("migr_eirt_ass"))

map <- gisco_get_nuts(
  resolution = "60",
  nuts_level = "0",
  year = "2016"
)

Linking to GEOS 3.6.2, GDAL 2.2.3, PROJ 4.9.3



# Non-EU citizens subject to the enforcement of immigration legislation in EU Member States, 2009-2019

In [4]:
refused <- ref_tot[geo == "EU28" & reason == "TOTAL" & indic_mg == "TOT_REF" & !(citizen %in% c("TOTAL","EUR_OTH","AME","AFR","ASI","EUR","OCE")), sum(values, na.rm = T),time]

apprehended <- app_tot[geo == "EU28" & !(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL" & sex == "T", sum(values, na.rm = T),time]

ordered <- ord_tot[geo == "EU28" & sex == "T" & !(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL", sum(values, na.rm =T ),time]

returned <- ret_tot[geo == "EU28" & sex == "T" & !(citizen %in% c("TOTAL","EUR_C_E_OTH","EUR_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL" & indic_mg == "TOT_RET", sum(values, na.rm =T ),time]

dt <- copy(refused)
dt[,apprehended := apprehended[,V1]]
dt[,ordered := ordered[1:8,V1]]
dt[,returned := returned[1:8,V1]]
setnames(dt,"V1","refused")

plot_ly(dt, x = ~time,y= ~refused,
               name = "Refused",type = "scatter",mode = 'lines+markers') %>%
  add_trace(y = ~apprehended, name = 'Apprehended', mode = 'lines+markers') %>%
  add_trace(y = ~ordered, name = 'Ordered to leave', mode = 'lines+markers') %>%
  add_trace(y = ~returned, name = 'Returned', mode = 'lines+markers') %>%
  layout(paper_bgcolor='gray20',
           plot_bgcolor='gray87',
           xaxis = list(color = 'black',tickangle = 0),
           yaxis = list(color = 'black', title = "Persons")
    ) %>%
  layout(hovermode = "x unified")


ERROR: Error in `[.data.table`(dt, , `:=`(ordered, ordered[1:8, V1])): Supplied 8 items to be assigned to 7 items of column 'ordered'. If you wish to 'recycle' the RHS please use rep() to make this intent clear to readers of your code.


# Non-EU citizens found to be illegally present 

## Overview 
### Map

In [ ]:
tmap_mode("view")
app_map <- app_tot[time == "2020-01-01" & !(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL" & sex == "T", sum(values, na.rm = T),geo]
setnames(app_map,"V1","Apprehended")

map <- left_join(map, app_map, by="geo")%>%
  tm_shape() +
  tm_fill("Apprehended",
    popup.vars = c("Apprehended","NUTS_ID", "NUTS_NAME"),
    palette = "Oranges", 
    breaks = c(0,1000, 5000, 25000, 50000, Inf),
    title = "Persons found to be illegally present"
  ) +
  tm_borders() +
  tm_layout(bg.color = "lightblue")

map

### Chart

In [ ]:
app <- app_tot[!(geo %in% c("EU27_2020","EU28")) & !(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL" & sex == "T",]
app[!(geo %in% c("DE","EL","ES","FR","HU")), geo := "OTH_MS"]
app <- app[,sum(values,na.rm = T), .(geo, time)]
app <- dcast(app, time~geo, value.var = "V1")

plot_ly(app, x = ~time,y= ~DE,
               name = "Germany",type = "scatter",mode = 'lines+markers') %>%
  add_trace(y = ~EL, name = 'Greece', mode = 'lines+markers') %>%
  add_trace(y = ~ES, name = 'Spain', mode = 'lines+markers') %>%
  add_trace(y = ~FR, name = 'France', mode = 'lines+markers') %>%
  add_trace(y = ~HU, name = 'Hungary', mode = 'lines+markers') %>%
  add_trace(y = ~OTH_MS, name = 'Other Member States', mode = 'lines+markers') %>%
  layout(paper_bgcolor='gray20',
           plot_bgcolor='gray87',
           xaxis = list(color = 'black',tickangle = 0),
           yaxis = list(color = 'black', title = "Persons")
    ) %>%
  layout(hovermode = "x unified")

## Non-EU citizens found to be illegally present — by sex and age{.tabset .tabset-fade .tabset-pills}
### Bar chart

In [ ]:
app_bar <- app_tot[geo == "EU28" & !(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & !(age %in% c("TOTAL","UNK","Y_LT18")) & sex != "T" & time == "2019-01-01", sum(values, na.rm = T),.(sex,age)] %>% 
  dcast( age~sex, value.var = "V1")

app_bar[age == "Y18-34", age := '18-34 years']
app_bar[age == "Y14-17", age := '14-17 years']
app_bar[age == "Y_LT14", age := '< 14 years']
app_bar[age == "Y_GE35", age := '>= 35 years']
app_bar <- app_bar[c(3,2,1,4)]


tit3 <- "Non-EU citizens found to be illegally present in the EU-27, by sex and age - 2019"
plot_ly(app_bar, x = ~F, y = ~age, type = 'bar', orientation = 'h', name = 'Women',
        marker = list(color = 'gold',
                      line = list(color = 'gold',
                                  width = 3)))%>%
  add_trace(x = ~M, name = 'Men',
            marker = list(color = 'darkblue',
                          line = list(color = 'darkblue',
                                      width = 3)))%>%
  layout(barmode = 'stack',
         xaxis = list(title = "Number of persons"),
         yaxis = list(title ="Age"),
         title=tit3)

### Table

**Top 20 countries of citizenship of non-EU citizens found to be illegally present in the EU-27, 2014-2019**

In [ ]:
app_tab <- app_tot[!(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL" & sex == "T" & time <= "2020-01-01" & time >= "2015-01-01",sum(values,na.rm = T),.(citizen,time) ]

top20 <- app_tab[,sum(V1,na.rm=T),citizen]
setnames(top20, "V1","Total")
top20 <- top20[order(-Total)]
top20 <- top20[1:20,]

app_tab[,time:=substring(time,1,4)]
app_tab <-  dcast(app_tab, citizen~time, value.var = "V1")

app_tab <- left_join(top20, app_tab)

datatable(app_tab[1:20], rownames = FALSE, filter="top", options = list(pageLength = 5, scrollX=T) )

# Non-EU citizens ordered to leave the EU-27
## Table 
**Non-EU citizens ordered to leave the territory of an EU Member State, the United Kingdom or an EFTA country, 2015-2020**

In [ ]:
ord_tab <-  ord_tot[sex == "T" & citizen == "TOTAL" & age == "TOTAL"& time >= "2015-01-01" ,sum(values,na.rm=T),.(geo,time)]
ord_tab[,time:=substring(time,1,4)]
ord_tab <-  dcast(ord_tab, geo~time, value.var = "V1")
datatable(ord_tab, rownames = FALSE, filter="top", options = list(pageLength = 5, scrollX=T) )

## Bar chart
**Top 20 countries of citizenship of non-EU citizens ordered to leave the EU-27, 2020 and 2019** 

In [ ]:
ord_tab <-  ord_tot[geo == "EU28" & sex == "T" & age == "TOTAL"& !(citizen %in% c("TOTAL","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & time <= "2019-01-01" & time >= "2018-01-01",sum(values,na.rm = T),.(citizen,time)]
ord_tab[,time:=substring(time,1,4)]
top20 <- ord_tab[time == "2019",]
top20 <- top20[order(-V1),]
top20 <- top20[1:20, .(citizen)]

ord_tab <- left_join(top20,ord_tab)
ord_tab <-  dcast(ord_tab, citizen~time, value.var = "V1")
setnames(ord_tab, names(ord_tab), c("citizen","year_1","year"))

plot_ly(ord_tab, x = ~citizen, y = ~year_1, type = 'bar', name = '2019')%>% 
  add_trace(y = ~year, name = '2020')%>%
  layout(barmode = 'group',
         xaxis = list(title = "Citizenship"),
         yaxis = list(title ="Number of persons"),
         hovermode='x unified')


# Returns of non-EU citizens
## Overview

In [ ]:
ret_tab <-  ret_tot[geo == "EU28" & sex == "T" & !(citizen %in% c("TOTAL","EUR_C_E_OTH","EUR_OTH","AME","AFR","ASI","EUR","OCE")) & age == "TOTAL" & indic_mg == "RET_THRD" & (time =="2019-01-01" | time == "2018-01-01"), sum(values, na.rm =T ),.(citizen,time)]
ret_tab[,time:=substring(time,1,4)]
top20 <- ret_tab[time == "2019",]
top20 <- top20[order(-V1),]
top20 <- top20[1:20, .(citizen)]

ret_tab <- left_join(top20,ret_tab)
ret_tab <-  dcast(ret_tab, citizen~time, value.var = "V1")
setnames(ret_tab, names(ret_tab), c("citizen","year_1","year"))

plot_ly(ret_tab, x = ~citizen, y = ~year_1, type = 'bar', name = '2018')%>% 
  add_trace(y = ~year, name = '2019')%>%
  layout(barmode = 'group',
         xaxis = list(title = "Citizenship"),
         yaxis = list(title ="Number of persons"),
         hovermode='x unified')

## Types of returns and assistance received

In [ ]:
ret_tab2 <-  ret_type[!(citizen %in% c("TOTAL","EUR_C_E_OTH","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & (time =="2020-01-01"), sum(values, na.rm =T ),.(geo,indic_mg)]

ret_tab2 <-  dcast(ret_tab2, geo~indic_mg, value.var = "V1")
ret_tab2 <- ret_tab2[,lapply(.SD, function(x) round(x/TOT_RET*100)), .SDcols=c("ENFORCE","OTH","VOLUNT"), by = geo]

plot_ly(ret_tab2, x = ~geo, y = ~ENFORCE, type = 'bar', name = 'Enforced return')%>% 
    add_trace(y = ~OTH, name = 'Other')%>%
  add_trace(y = ~VOLUNT, name = 'Voluntary return')%>%
    layout(barmode = 'stack',
           xaxis = list(title = ""),
           yaxis = list(title ="%"),
           hovermode='x unified')


In [ ]:
ret_tab3 <-  ret_ass[!(citizen %in% c("TOTAL","EUR_C_E_OTH","EUR_C_E_OTH","AME","AFR","ASI","EUR","OCE")) & (time =="2019-01-01"), sum(values, na.rm =T ),.(geo,indic_mg)]

ret_tab3 <-  dcast(ret_tab3, geo~indic_mg, value.var = "V1")
ret_tab3 <- ret_tab3[,lapply(.SD, function(x) round(x/TOT_RET*100)), .SDcols=c("ASSIST","N_ASSIST","UNK"), by = geo]

plot_ly(ret_tab3, x = ~geo, y = ~ASSIST, type = 'bar', name = 'Assisted return')%>% 
    add_trace(y = ~N_ASSIST, name = 'Non assisted return')%>%
  add_trace(y = ~UNK, name = 'Unkown')%>%
    layout(barmode = 'stack',
           xaxis = list(title = ""),
           yaxis = list(title ="%"),
           hovermode='x unified')

# Non-EU citizens refused entry into the EU-27
## Overview

In [ ]:
ref_pie <- ref_tot[geo !="EU27_2020" & time == "2020-01-01" & reason == "TOTAL" & indic_mg == "TOT_REF" ,.(geo,values) ]
plot_ly( data = ref_pie, labels = ~geo, values = ~values, type = "pie", textinfo = "label+percent", insidetextorientation = "radial" ) 

## Bar charts

In [ ]:
ref_tab <-  ref_tot[!(geo %in% c("EU28","EU27_2020")) &reason == "TOTAL" & !(citizen %in% c("TOTAL","EUR_OTH","AME","AFR","ASI","EUR","OCE")) & time == "2019-01-01", sum(values, na.rm = T), .(citizen, indic_mg)]

top20 <- ref_tab[indic_mg=="TOT_REF"]
top20 <- top20[order(-V1)]
top20 <- top20[1:20,.(citizen)]

ref_tab <- left_join(top20, ref_tab)
ref_tab <-  dcast(ref_tab, citizen~indic_mg, value.var = "V1")

plot_ly(ref_tab, x = ~citizen, y = ~REF_AIR, type = 'bar', name = 'Air')%>% 
    add_trace(y = ~REF_LAND, name = 'Land')%>%
  add_trace(y = ~REF_SEA, name = 'Sea')%>%
    layout(barmode = 'stack',
           xaxis = list(title = ""),
           yaxis = list(title ="%"),
           hovermode='x unified')

In [ ]:
ref_tab2 <-  ref_tot[!(geo %in% c("EU28","EU27_2020")) &reason != "TOTAL" &indic_mg == "TOT_REF" & !(citizen %in% c("TOTAL","EUR_OTH","AME","AFR","ASI","EUR","OCE")) & time == "2019-01-01", sum(values, na.rm = T), .(reason)]

plot_ly(ref_tab2, x = ~reason, y = ~V1, type = 'bar', name = 'Reason')%>% 
    layout(barmode = 'stack',
           xaxis = list(title = ""),
           yaxis = list(title ="Number"),
           hovermode='x unified')